<a href="https://colab.research.google.com/github/whart222/pao/blob/master/doc/besancon27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A PAO Example: Solving Besancon27**

NOTE: This notebook assumes that PAO and related solvers are installed in the python environment that is used to launch jupyter.

## Creating and Solving a PAO Model using Pyomo

In [2]:
import pyomo.environ as pe
from pao.pyomo import *

Example 2.7 from

    Near-Optimal Robust Bilevel Optimization
    M. Besancon, M. F. Anjos and L. Brotcorne
    arXiv:1908.04040v5 (2019)

Optimal solution: (x,v) = (0,1)

In [3]:
M = pe.ConcreteModel()

M.x = pe.Var(bounds=(0.5,None),initialize=1)  # Lower bound set to 0.5
M.v = pe.Var(initialize=1)

M.o = pe.Objective(expr=M.x, sense=pe.minimize)
M.c = pe.Constraint(expr= M.v >= 1 - M.x/10)

M.L = SubModel(fixed=M.x)
M.L.o = pe.Objective(expr=M.v, sense=pe.maximize)
M.L.c = pe.Constraint(expr= 1 + M.x/10 >= M.v)

Show available solvers

In [4]:
Solver.summary()

pao.mpr.FA
    PAO solver for Multilevel Problem Representations that define linear
    bilevel problems.  Solver uses big-M relaxations discussed by Fortuny-
    Amat and McCarl (1981).

pao.mpr.PCCG
    PAO solver for Multilevel Problem Representations that define linear
    bilevel problems. Solver uses projected column constraint generation
    algorithm described by Yue et al. (2017).

pao.mpr.REG
    PAO solver for Multilevel Problem Representations that define linear
    bilevel problems.  Solver uses regularization discussed by Scheel and
    Scholtes (2000) and Ralph and Wright (2004).

pao.mpr.interdiction
    PAO solver for Multilevel Problem Representations that define linear
    interdiction problems, where the upper- and lower-objectives are
    opposite.

pao.pyomo.FA
    PAO solver for Pyomo models that define linear and bilinear bilevel
    problems.  Solver uses big-M relaxations discussed by Fortuny-Amat and
    McCarl (1981).

pao.pyomo.PCCG
    PAO solver for Pyomo

Apply solver

In [5]:
solver = Solver("pao.pyomo.FA")
results = solver.solve(M, mip_solver="glpk")
# results = solver.solve(M, mip_solver=Solver("glpk", mipgap=0.5), nlp_solver=Solver("ipopt"))
# results = solver.solve(M, mip_solver=Solver("glpk"))
# results = solver.solve(M, mip_solver=Solver("neos.glpk", mipgap=0.5))

The results object contains information about the problem and solver

In [6]:
print(results)

Problem:
- lower_bound: None
  name: 'unknown'
  number_of_binary_variables: None
  number_of_constraints: None
  number_of_continuous_variables: None
  number_of_integer_variables: None
  number_of_objectives: None
  number_of_variables: None
  sense: None
  upper_bound: None
Solver:
- best_feasible_objective: 0.5
  config: <pyomo.common.config.ConfigDict object at 0x7f5c496e2b50>
  lmp_solver: 'pao.mpr.FA'
  name: 'pao.pyomo.FA'
  rc: None
  solver_time: None
  termination_condition: <TerminationCondition.optimal: 'optimal'>
  wallclock_time: 0.30992722511291504


The solution is loaded back into the model

In [7]:
print("v", M.v.value)
print("x", M.x.value)

v 0.94999999999709
x 0.5
